## Yield Spread model
Adapted by Developer on 07/19/2022

This notebook implements a model to predict yield spreads from reference and trade history data. The model uses an attention layer between the two LSTM layers. Compared to the original model, this one takes calc date as a categorical input, and the log10 durations to each possible calc-date, in days, as numerical features

Last modification: Experimenting with different target trade features

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import time

import numpy as np
from google.cloud import bigquery
from google.cloud import storage
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns


from tensorflow.keras.layers import Embedding
from tensorflow.keras import activations
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn import preprocessing
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMRegressor
import lightgbm

from IPython.display import display, HTML
import os

import wandb
from wandb.keras import WandbCallback


from ficc.data.process_data import process_data
from ficc.utils.auxiliary_variables import PREDICTORS, NON_CAT_FEATURES, BINARY, CATEGORICAL_FEATURES, IDENTIFIERS, PURPOSE_CLASS_DICT
from ficc.utils.gcp_storage_functions import upload_data, download_data

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
tf.__version__

'2.8.0'

Setting the environment variables

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="eng-reactor-287421-112eb767e1b3.json"
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.options.mode.chained_assignment = None

Initializing BigQuery client

In [4]:
bq_client = bigquery.Client()

Initializing GCP storage client

In [5]:
storage_client = storage.Client()

Declaring hyper-parameters

In [6]:
TRAIN_TEST_SPLIT = 0.85
LEARNING_RATE = 0.0001
BATCH_SIZE = 1000
NUM_EPOCHS = 100

DROPOUT = 0.01
SEQUENCE_LENGTH = 5
NUM_FEATURES = 5

#### Initializing Wandb

In [7]:
# wandb.init(project="yield_spread_model", entity="ficc-ai", name="window_train_till_may_2022_test_may_2022")

#### Query to fetch data

We create the training data from the trades which occurred between August 2021 and June 2022. All three trade directions, namely dealer-dealer (D), dealer-sells (S), and dealer-purchases (P) are included. We are limiting the training to bonds whose yield is a positive. The maturity description code is restricted to 2, to remove all muni derivatives.



In [8]:
DATA_QUERY = ''' 
SELECT
  * 
FROM
  `eng-reactor-287421.auxiliary_views.materialized_trade_history`
WHERE
  yield IS NOT NULL
  AND yield > 0
  AND par_traded >= 10000
  AND trade_date >= '2021-10-01'
  AND maturity_description_code = 2
  AND coupon_type in (8, 4, 10)
  AND capital_type <> 10
  AND default_exists <> TRUE
  AND sale_type <> 4
  AND sec_regulation IS NULL
  AND most_recent_default_event IS NULL
  AND default_indicator IS FALSE
  --AND DATETIME_DIFF(trade_datetime,recent[SAFE_OFFSET(0)].trade_datetime,SECOND) < 1000000 -- 12 days to the most recent trade
  AND msrb_valid_to_date > current_date -- condition to remove cancelled trades
ORDER BY
  trade_datetime DESC ''' 

#### Data Preparation
The trade history is an array which contains the yield spread, trade type, trade size, and the number of seconds ago the trade occured. 

We grab the data from a GCP bucket

In [9]:
import gcsfs
fs = gcsfs.GCSFileSystem(project='eng-reactor-287421')
with fs.open('ficc_training_data_latest/processed_data.pkl') as f:
    data = pd.read_pickle(f)

Here is a list of what we are currently excluding: 
<ul>
    <li>Variable coupon</li>
    <li>Derivatives</li>
    <li>Zero coupon bonds</li>
    <li>Term bonds </li>
    <li>Territories (VI, GU, PR)</li>
    <li>Called bonds</li>
    <li>Crossover refunding and partially pre-refunded</li>
    <li>Maturity less than a year in the future and more than 30 years in the future</li>
    <li>Callable less than a year in the future </li>
    <li>Restructured debt</li>
    <li>Defaulted securities</li>
    <li>Private placement/subject to SEC regulation 144A</li>
    <li>Purpose Types:</li>
    <ul>
        <li>Assisted_living</li>
        <li>Continuing Care Retirement Center</li>
        <li>Correctional facilities</li>
        <li>Harbor/chanel</li>
        <li>Mall</li>
        <li>Real Estate</li>
    </ul>
<ul>

In [10]:
data = data[(data.days_to_call == 0) | (data.days_to_call > np.log10(400))]
data = data[(data.days_to_refund == 0) | (data.days_to_refund > np.log10(400))]
data = data[data.days_to_maturity < np.log10(30000)]
data = data[data.sinking == False]
data = data[data.incorporated_state_code != 'VI']
data = data[data.incorporated_state_code != 'GU']
data = data[(data.coupon_type == 8)]
data = data[data.is_called == False]
data = data[~data.purpose_sub_class.isin([6, 20, 22, 44, 57, 90])]
data = data[~data.called_redemption_type.isin([18, 19])]

### Adding target trade features to calculate attention

As a first step, we only utilize the size of the trade and the directions as features to calculate the attention. Going forward we will be adding more features like the state code, coupon, interest payment frequency, etc. 

In [11]:
trade_mapping = {'D':[0,0], 'S':[0,1], 'P':[1,0]}
def target_trade_processing_for_attention(row):
    target_trade_features = []
    target_trade_features.append(row['quantity'])
    target_trade_features = target_trade_features + trade_mapping[row['trade_type']]
    #target_trade_features.append(row['coupon'])
    return np.tile(target_trade_features, (5,1))

In [12]:
%%time
data['target_attention_features'] = data.parallel_apply(target_trade_processing_for_attention, axis = 1)

CPU times: user 19.7 s, sys: 4.65 s, total: 24.3 s
Wall time: 41 s


In [13]:
data.iloc[0]['target_attention_features']

array([[4.87506104, 0.        , 0.        ],
       [4.87506104, 0.        , 0.        ],
       [4.87506104, 0.        , 0.        ],
       [4.87506104, 0.        , 0.        ],
       [4.87506104, 0.        , 0.        ]])

For the purpose of plotting, not used in training

In [14]:
data.purpose_sub_class.fillna(0, inplace=True)

#### Replacing the ratings with the stand alone ratings. This is done to exclude enhancements. 

In [15]:
data.loc[data.sp_stand_alone.isna(), 'sp_stand_alone'] = 'NR'

For some versions of pandas you cannot intorduce new categories in a colum with dtype categorical. Thus changing it to string

In [16]:
data.rating = data.rating.astype('str')
data.sp_stand_alone = data.sp_stand_alone.astype('str')

In [17]:
data.loc[(data.sp_stand_alone != 'NR'),'rating'] = data[(data.sp_stand_alone != 'NR')]['sp_stand_alone'].loc[:]

In [18]:
data[(data.sp_stand_alone != 'NR')][['rating','sp_stand_alone','sp_long']]

,rating,sp_stand_alone,sp_long
20,A+,A+,AA
21,A+,A+,AA
38,A+,A+,AA
39,A+,A+,AA
40,A+,A+,AA
...,...,...,...
5010851,A+,A+,AA
5010863,A+,A+,AA
5010889,BBB+,BBB+,AA
5010890,BBB+,BBB+,AA


### Add duration to all possible calc dates, as inputs to the model

In [19]:
for date_type in ['next_call', 'par_call', 'maturity', 'refund']:
    data.loc[:, date_type + '_duration'] = (data[date_type + '_date'] - data.trade_date).dt.days
    data.loc[:, date_type + '_duration'].fillna(-1.0, inplace=True)

Selecting a subset of features that will be used to train on.

In [20]:
if 'target_attention_features' not in PREDICTORS:
    PREDICTORS.append('target_attention_features')
if 'calc_day_cat' not in CATEGORICAL_FEATURES:
    CATEGORICAL_FEATURES.append('calc_day_cat')
# for date_type in ['next_call', 'par_call', 'maturity', 'refund']:
#     if date_type + '_duration' not in PREDICTORS:
#         PREDICTORS.append(date_type + '_duration')
#     if date_type + '_duration' not in NON_CAT_FEATURES:
#         NON_CAT_FEATURES.append(date_type + '_duration')

In [ ]:
processed_data = data[IDENTIFIERS + PREDICTORS + ['dollar_price','calc_date', 'trade_date','trade_datetime', 'purpose_sub_class', 'called_redemption_type', 'muni_issue_type','yield','ficc_ycl']]

In [ ]:
len(processed_data)

In [ ]:
np.unique(processed_data.rating)

#### Fitting encoders to the categorical features. These encoders are then used to encode the categorical features of the train and test set

In [ ]:
encoders = {}
fmax = {}
for f in CATEGORICAL_FEATURES:
    print(f)
    fprep = preprocessing.LabelEncoder().fit(processed_data[f].drop_duplicates())
    fmax[f] = np.max(fprep.transform(fprep.classes_))
    encoders[f] = fprep
    
with open('encoders.pkl','wb') as file:
    pickle.dump(encoders,file)

In [ ]:
upload_data(storage_client, 'ficc_training_data_latest', f"encoders.pkl")

#### Splitting the data into train and test sets



In [ ]:
len(processed_data)

In [ ]:
train_dataframe = processed_data[(processed_data.trade_date < '07-01-2022')]

In [ ]:
test_dataframe = processed_data[(processed_data.trade_date >= '07-01-2022')] # & (processed_data.trade_date <= '12-31-2021')]

In [ ]:
len(train_dataframe)

In [ ]:
len(test_dataframe)

Converting data into format suitable for the model

In [ ]:
def create_input(df):
    global encoders
    datalist = []
    datalist.append(np.stack(df['trade_history'].to_numpy()))
    datalist.append(np.stack(df['target_attention_features'].to_numpy()))

    noncat_and_binary = []
    for f in NON_CAT_FEATURES + BINARY:
        noncat_and_binary.append(np.expand_dims(df[f].to_numpy().astype('float32'), axis=1))
    datalist.append(np.concatenate(noncat_and_binary, axis=-1))

    for f in CATEGORICAL_FEATURES:
        encoded = encoders[f].transform(df[f])
        datalist.append(encoded.astype('float32'))
        
    return datalist

In [ ]:
%%time
x_train = create_input(train_dataframe)
y_train = train_dataframe.yield_spread

In [ ]:
%%time
x_test = create_input(test_dataframe)
y_test = test_dataframe.yield_spread

In [ ]:
x_train[1].shape

In [ ]:
x_test[2].shape

#### Adapting Normalization layers to the non categorical features

In [ ]:
# Normalization layer for the trade history
trade_history_normalizer = Normalization(name='Trade_history_normalizer')
trade_history_normalizer.adapt(x_train[0])

In [ ]:
# Normalization layer for the non-categorical and binary features
noncat_binary_normalizer = Normalization(name='Numerical_binary_normalizer')
noncat_binary_normalizer.adapt(x_train[2])

#### Setting the seed for intialization of the layers

In [ ]:
tf.keras.utils.set_random_seed(10)

#### Attention layer
This is an implementation of a layer that calculates scaled dot product attention. 

In [ ]:
class CustomAttention(tf.keras.layers.Layer):
    def __init__(self, depth):
        super(CustomAttention, self).__init__()
        self.depth = depth
        self.wq = layers.Dense(depth, name='weights_query') 
        self.wk = layers.Dense(depth, name='weights_key')
        self.wv = layers.Dense(depth, name='weights_value')

    def scaled_dot_product_attention(self, q, v, k):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        scaling = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(scaling)
        
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=1) 
        output = tf.matmul(attention_weights, v)
        
        return output
    
    def call(self, q, v, k):
        
        q = self.wq(q)
        v = self.wv(v)
        k = self.wk(k)

        output = self.scaled_dot_product_attention(q, v, k)
        
        return output    

#### Implementation of the model

In [ ]:
inputs = []
layer = []

############## INPUT BLOCK ###################
trade_history_input = layers.Input(name="trade_history_input", 
                                   shape=(SEQUENCE_LENGTH,NUM_FEATURES), 
                                   dtype = tf.float32) 

target_attention_input = layers.Input(name="target_attention_input", 
                                   shape=(SEQUENCE_LENGTH, 3), 
                                   dtype = tf.float32) 


inputs.append(trade_history_input)
inputs.append(target_attention_input)

inputs.append(layers.Input(
    name="NON_CAT_AND_BINARY_FEATURES",
    shape=(len(NON_CAT_FEATURES + BINARY),)
))


layer.append(noncat_binary_normalizer(inputs[2]))
####################################################


############## TRADE HISTORY MODEL #################

# Adding the time2vec encoding to the input to transformer
lstm_layer = layers.LSTM(50, 
                         activation='tanh',
                         input_shape=(SEQUENCE_LENGTH,NUM_FEATURES),
                         return_sequences = True,
                         name='LSTM')

# lstm_attention_layer = layers.Attention(use_scale=True, name='attention_layer_1')
lstm_attention_layer = CustomAttention(50)

lstm_layer_2 = layers.LSTM(100, 
                           activation='tanh',
                           input_shape=(SEQUENCE_LENGTH,50),
                           return_sequences = False,
                           name='LSTM_2')


features = lstm_layer(trade_history_normalizer(inputs[0]))
# features = lstm_attention_layer([features, features])
features = lstm_attention_layer(features, features, inputs[1])
features = layers.BatchNormalization()(features)
# features = layers.Dropout(DROPOUT)(features)

features = lstm_layer_2(features)
features = layers.BatchNormalization()(features)
# features = layers.Dropout(DROPOUT)(features)

trade_history_output = layers.Dense(100, 
                                    activation='relu')(features)

####################################################

############## REFERENCE DATA MODEL ################
global encoders
for f in CATEGORICAL_FEATURES:
    fin = layers.Input(shape=(1,), name = f)
    inputs.append(fin)
    embedded = layers.Flatten(name = f + "_flat")( layers.Embedding(input_dim = fmax[f]+1,
                                                                    output_dim = max(30,int(np.sqrt(fmax[f]))),
                                                                    input_length= 1,
                                                                    name = f + "_embed")(fin))
    layer.append(embedded)

    
reference_hidden = layers.Dense(400,
                                activation='relu',
                                name='reference_hidden_1')(layers.concatenate(layer, axis=-1))

reference_hidden = layers.BatchNormalization()(reference_hidden)
reference_hidden = layers.Dropout(DROPOUT)(reference_hidden)

reference_hidden2 = layers.Dense(200,activation='relu',name='reference_hidden_2')(reference_hidden)
reference_hidden2 = layers.BatchNormalization()(reference_hidden2)
reference_hidden2 = layers.Dropout(DROPOUT)(reference_hidden2)

reference_output = layers.Dense(100,activation='tanh',name='reference_hidden_3')(reference_hidden2)

####################################################

feed_forward_input = layers.concatenate([reference_output, trade_history_output])

hidden = layers.Dense(300,activation='relu')(feed_forward_input)
hidden = layers.BatchNormalization()(hidden)
hidden = layers.Dropout(DROPOUT)(hidden)

hidden2 = layers.Dense(100,activation='tanh')(hidden)
hidden2 = layers.BatchNormalization()(hidden2)
hidden2 = layers.Dropout(DROPOUT)(hidden2)

final = layers.Dense(1)(hidden2)

model = keras.Model(inputs=inputs, outputs=final)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model,
    show_shapes=False,
    show_layer_names=True,
    rankdir="LR",
    expand_nested=False,
    dpi=96,
)

In [ ]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

In [ ]:
fit_callbacks = [
    #WandbCallback(save_model=False),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        verbose=0,
        mode="auto",
        restore_best_weights=True,
    ),
    time_callback
]

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
          loss=keras.losses.MeanAbsoluteError(),
          metrics=[keras.metrics.MeanAbsoluteError()])

In [ ]:
%%time 
history = model.fit(x_train, 
                    y_train, 
                    epochs=100, 
                    batch_size=BATCH_SIZE, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=fit_callbacks,
                    use_multiprocessing=True,
                    workers=8)

#### Plotting train vs validation loss

In [ ]:
plt.plot(range(len(history.history['val_loss'])),history.history['val_loss'], label='val_loss')
plt.plot(range(len(history.history['loss'])),history.history['loss'], label='loss')
plt.title('Validation loss and training Loss per epoch')
plt.legend(loc="upper right")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

#### Gigaflops for one epoch of training

In [ ]:
avg_time = np.mean(time_callback.times)

In [ ]:
n = len(x_train[0])
p = model.count_params()
avg_time = np.mean(time_callback.times)
gflops = ((n*p*2*3)/avg_time)/10**9

print(gflops)

### Test accuracy on the entire test set

In [ ]:
len(test_dataframe)

In [ ]:
_, mae = model.evaluate(x_test, y_test, verbose=1)
print(f"Test loss: {round(mae, 3)}")
# wandb.log({"Test MAE": mae})

### Accuracy on a daily basis

Measuring the daily accuracy for large dealer dealer trades

In [ ]:
for d in [d for d in pd.date_range(start="06/01/2022",end="07/31/2022",freq='D')]:
    next_day = test_dataframe[(test_dataframe.trade_date == d) &
                              (test_dataframe.trade_type == 'D') &
                              (test_dataframe.quantity >= np.log10(500000)) & 
                              (test_dataframe.coupon == 5) ].copy()
    if len(next_day) == 0:
        continue
    next_day_test = create_input(next_day)  
    next_day_preds = model.predict(next_day_test)
    error = next_day.yield_spread - next_day_preds.reshape(-1)
    MAE = np.mean(np.abs(error))
    print(f"Date :{d.date()} MAE:{MAE}" )

### Test accuracy on large dealer-dealer trades
We define large as any trade which is above $500,000

In [ ]:
true_mid = test_dataframe[(test_dataframe.trade_type == 'D') & (test_dataframe.quantity >= np.log10(500000)) & (test_dataframe.coupon == 5)]

In [ ]:
len(true_mid)

In [ ]:
# true_mid.value_counts('rating')

In [ ]:
# true_mid = true_mid[(true_mid.rating == 'AAA') | (true_mid.rating == 'MR')]# & (true_mid.muni_issue_type != 14)]

In [ ]:
# true_mid = true_mid[true_mid.trade_date > '12-15-2021']

In [ ]:
len(true_mid)

In [ ]:
%%time
x_true_mid = create_input(true_mid)
y_true_mid = true_mid.yield_spread

In [ ]:
_, mae = model.evaluate(x_true_mid, y_true_mid, verbose=1)
print(f"Test MAE: {round(mae, 3)}")
# wandb.log({"Dealer Dealer true mid Test MAE": mae})

### Alternative evaluation set
Experiment with other conditions. From the error analysis notebook, it was observed that the number of accrued days that have passed since the beginning of interest being accrued is important

In [ ]:
temp_test = test_dataframe[(test_dataframe['coupon'] == 5)]

In [ ]:
len(temp_test)

In [ ]:
%%time
x_temp_test = create_input(temp_test)
y_temp_test = temp_test.yield_spread

In [ ]:
_, mae = model.evaluate(x_temp_test, y_temp_test, verbose=1)
print(f"Test MAE: {round(mae, 3)}")

# Saving the model

In [ ]:
tf.get_logger().setLevel('ERROR')

In [ ]:
save_model = False

if save_model:
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    file_timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M')
    print(f"file time stamp : {file_timestamp}")

    print("Saving encoders and uploading encoders")
    with open(f"encoders_{file_timestamp}.pkl",'wb') as file:
        pickle.dump(encoders,file)    
    upload_data(storage_client, 'ficc_encoders', f"encoders_{file_timestamp}.pkl")

    print("Saving and uploading model")
    model.save(f"saved_model_calc_date_conditioned_june_{file_timestamp}")
    import shutil
    shutil.make_archive(f"model_calc_date_conditioned_jul", 'zip', f"saved_model_calc_date_conditioned_june_{file_timestamp}")
    upload_data(storage_client, 'ficc_training_data_latest', f"model_calc_date_conditioned_jul.zip")

# Evaluate the ability for a calc-date conditioned model to determine the final price (TODO)